# Universal Cloud Bridge (Colab + Kaggle -> Local .NET)

This notebook turns Colab into an **AI Server** that:
1.  Connects to **Kaggle** to download datasets.
2.  Connects to your **Local .NET App** via **Ngrok** (Static Domain).
3.  **Runs Training Simulations** and **Returns Results**.

## 1. Setup Environment

In [ ]:
!pip install flask flask-ngrok pyngrok easyocr torch torchvision torchaudio kaggle
!pip install tensorflow

## 2. Authenticate Kaggle (Data Source)
**Step 1:** Upload your `kaggle.json` file to the Colab files on the left.

In [ ]:
import os

def setup_kaggle():
    if not os.path.exists('kaggle.json'):
        print("❌ ERROR: Please upload kaggle.json to the file explorer first!")
        return False

    # Move to correct location
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("✅ Kaggle Configured Successfully!")
    return True

setup_kaggle()

## 3. Authenticate Ngrok (Tunnel)
Get token from: [dashboard.ngrok.com](https://dashboard.ngrok.com)

In [ ]:
from pyngrok import ngrok

# REPLACE WITH YOUR TOKEN HERE
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"

ngrok.set_auth_token(NGROK_TOKEN)

## 4. Define AI Logic

In [ ]:
import easyocr
import json
import time
import random

# Initialize Models
print("Loading AI Models...")
reader = easyocr.Reader(['en'], gpu=True)
print("Models Loaded!")

def perform_ocr(text_or_path):
    return f"OCR Simulated Result for {text_or_path}"

def perform_training(data):
    print(f"--- Starting AI Model Training (In Cloud) ---")
    # Handle both single string and list of strings
    data_count = len(data) if isinstance(data, list) else (1 if data else 0)
    print(f"Training requested for data count: {data_count}")
    
    # 1. Download Dataset (Kaggle)
    print("Downloading Dataset from Kaggle...")
    # !kaggle datasets download -d ...
    time.sleep(1)
    
    # 2. Training Loop
    epochs = 5
    current_loss = 0.9
    
    for epoch in range(1, epochs + 1):
        time.sleep(0.5)
        current_loss *= 0.7
        accuracy = 1 - current_loss
        print(f" > Epoch {epoch}/{epochs}: Loss = {current_loss:.4f}, Accuracy = {accuracy:.4f}")
    
    print("--- Training Complete! ---")
    
    # Return metrics to be saved by the .NET App
    return {
        "status": "success", 
        "epochs": epochs, 
        "final_loss": round(current_loss, 4),
        "final_accuracy": round(1 - current_loss, 4),
        "message": "Model trained in Cloud on " + str(data_count) + " records."
    }

## 5. Start Server

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "online", "kaggle": os.path.exists('/root/.kaggle/kaggle.json')})

@app.route('/predict', methods=['POST'])
def predict():
    try:
        content = request.json.get('text', '')
        return jsonify({"result": f"Cloud Analysis for: {content}", "sentiment": "Positive"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/train', methods=['POST'])
def train():
    data = request.json
    result = perform_training(data)
    return jsonify(result)

@app.route('/train-all', methods=['POST'])
def train_all():
    content = request.json
    dataset = content.get('dataset', [])
    print(f"Received bulk training request with {len(dataset)} records.")
    
    # Extract text from the list of objects
    text_data = [item.get('text', '') for item in dataset]
    
    result = perform_training(text_data)
    return jsonify(result)

# Open Ngrok Tunnel using your STATIC DOMAIN
try:
    ngrok.kill()
    public_url = ngrok.connect(5000, domain="semijuridically-unwretched-carlota.ngrok-free.dev").public_url
    print(f"\n\n✅ SUCCESS! Server URL: {public_url}\n")
except Exception as e:
    print(f"Error starting ngrok: {e}")
    public_url = ngrok.connect(5000).public_url
    print(f"Fallback URL: {public_url}")

app.run(port=5000)